# Computational Finance - Assignment 1.3

---
---

This Jupyter notebook contains all the code for the first section of Assignment 1. The content is organized by topic, following the same structure as the accompanying paper.


In [1]:
import statistics
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

from scipy import stats

In [2]:
class BlackScholesModel:
    def __init__(self, T, S0, K, r, sigma, steps=1, sigma_val=None):
        self.T = T
        self.S0 = S0
        self.K = K
        self.r = r
        self.sigma = sigma
        if sigma_val is None:
            self.sigma_val = sigma
        else:
            self.sigma_val = sigma_val

        self.steps = steps
        self.dt = T / steps
        self.price = S0
        self.price_path = np.zeros(steps)

        self.delta_list = None
        self.x_hedge = None

    def call_price(self, t=0):
        """
        """
        d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma_val ** 2)
              * (self.T - t)) / (self.sigma_val * np.sqrt(self.T - t))
        d2 = d1 - self.sigma_val * np.sqrt(self.T - t)

        call = (self.S0 * st.norm.cdf(d1, 0.0, 1.0) - self.K *
                np.exp(-self.r * self.T) * st.norm.cdf(d2, 0.0, 1.0))

        return call

    def put_price(self, t=0):
        """
        """
        d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma_val ** 2)
              * (self.T - t)) / (self.sigma_val * np.sqrt(self.T - t))
        d2 = d1 - self.sigma_val * np.sqrt(self.T - t)

        put = ((self.K * np.exp(-self.r * self.T)
                * st.norm.cdf(-d2, 0.0, 1.0)) - self.S0 *
               st.norm.cdf(-d1, 0.0, 1.0))

        return put

    def generate_price_path(self):
        """
        """
        for i in range(self.steps):
            self.price_path[i] = self.price
            dS = self.r * self.price * self.dt + self.sigma * \
                self.price * np.random.normal(0, 1) * np.sqrt(self.dt)

            self.price += dS

    def create_hedge(self, steps=1, hedge_setting='Call'):
        '''
        Simulate hedging over the given price path and returns a profit
        '''
        # time steps
        x_hedge = [j / steps for j in range(steps)]

        # Check if price path is made
        if self.price_path[-1] == 0:
            self.generate_price_path()

        # corrected current price for hedge time intervals and all deltas for a given time
        hedge_price = [j for n, j in enumerate(self.price_path) if int(n % (self.steps / steps)) == 0]
        delta_list = [self.hedge(t, s, hedge_setting) for t, s in zip(x_hedge, hedge_price)]

        # New time step and interest for given interval
        dt = self.T / steps
        interest = self.r * dt

        # set iterables
        delta_t = 0
        current_stock_price = 0

        # set loop variables
        previous_delta = delta_t
        bank = self.call_price()

        # loop over the time step and hedge for every time step
        for delta_t, current_stock_price in zip(delta_list, hedge_price):
            cost = (delta_t - previous_delta) * current_stock_price
            bank = bank * np.exp(interest) - cost
            previous_delta = delta_t

        # Calculate the profit when t = T
        profit = bank + (current_stock_price * delta_t) - max([current_stock_price - self.K, 0])

        # Save values for later evaluations
        self.delta_list = delta_list
        self.x_hedge = x_hedge
        self.hedge_price = hedge_price
        
        # return profit made during the hedging
        return profit
    
    def plot_price_path(self, hedge_plot=True):
        '''
        Plot both price path and delta over time
        '''
        fig, ax1 = plt.subplots()
        
        # Get price time steps
        x_price = [i / self.steps for i in range(self.steps)]
        
        # Plot the price over time on the first axis
        color = 'tab:red'
        ax1.set_xlabel('years')
        ax1.set_ylabel('Price', color=color)
        ax1.plot(x_price, self.price_path, color=color,
                 label="Discritized Black Scholes")
        ax1.tick_params(axis='y', labelcolor=color)

        if hedge_plot:
            # Instantiate a second axes that shares the same x-axis
            ax2 = ax1.twinx()
            color = 'tab:blue'
            # we already handled the x-label with ax1
            ax2.set_ylabel('Delta', color=color)
            # Plot the delta
            ax2.plot(self.x_hedge, self.delta_list, color=color, label='Hedge delta')
            ax2.tick_params(axis='y', labelcolor=color)
           
        # Finalize the plot and show
        plt.title("Stock price and Delta development over time")
        fig.tight_layout()
        plt.show()

    def hedge(self, t, S, hedge_setting='call'):
        '''
        Calculate the delta at a given time
        '''
        # Take d1 from black-scholes
        d1 = (np.log(S / self.K) + (self.r + 0.5 * self.sigma_val ** 2)
              * (self.T - t)) / (self.sigma_val * np.sqrt(self.T - t))
        
        # Calculate derivitive for call and put
        if hedge_setting.lower() == 'call':
            return st.norm.cdf(d1, 0.0, 1.0)

        elif hedge_setting.lower() == 'put':
            return st.norm.cdf(d1, 0.0, 1.0) - 1
        else:
            print("Setting not found")
            return None


## Varying frequency of hedging from daily to weekly

In [3]:
def sim_hedging(steps, years, start_price, strike_price, rate, volatility, hedge_steps, n_sims=1000):
    profit=np.zeros(n_sims)
    for i in range(n_sims):
        B = BlackScholesModel(years, start_price, strike_price, rate, volatility, steps)
        profit[i] = B.create_hedge(hedge_steps)
    return profit

In [ ]:
'''Hedging Daily'''
profit_daily = sim_hedging(steps = 365,
    years = 1,
    start_price = 100,
    strike_price = 99,
    rate = 0.06,
    volatility = 0.2,
    hedge_steps=365)

'''Hedging Alternate Days'''
profit_alternative_days = sim_hedging(steps = 365,
    years = 1,
    start_price = 100,
    strike_price = 99,
    rate = 0.06,
    volatility = 0.2,
    hedge_steps=182)

'''Hedging Weekly'''
profit_weekly = sim_hedging(steps = 365,
    years = 1,
    start_price = 100,
    strike_price = 99,
    rate = 0.06,
    volatility = 0.2,
    hedge_steps=52)

'''Hedging Monthly'''
profit_monthly = sim_hedging(steps = 365,
    years = 1,
    start_price = 100,
    strike_price = 99,
    rate = 0.06,
    volatility = 0.2,
    hedge_steps=12)

'''Hedging Quaterly'''
profit_quaterly = sim_hedging(steps = 365,
    years = 1,
    start_price = 100,
    strike_price = 99,
    rate = 0.06,
    volatility = 0.2,
    hedge_steps=4)

In [ ]:
plt.figure(figsize=(16,8))
plt.hist(profit_daily, bins=50, alpha=1.0, color='green',
         label=f'Daily: $\mu={np.mean(profit_daily):.3f}, \sigma={np.std(profit_daily):.3f}$')
# plt.hist(profit_alternative_days, bins=50, alpha=0.6, color='green',
#          label=f'Alt. Days: $\mu={np.mean(profit_alternative_days):.3f}, \sigma={np.std(profit_alternative_days):.3f}$')
plt.hist(profit_weekly, bins=50, alpha=0.6, color='lightblue',
         label=f'Weekly: $\mu={np.mean(profit_weekly):.3f}, \sigma={np.std(profit_weekly):.3f}$')
plt.hist(profit_monthly, bins=50, alpha=0.4, color='orange',
         label=f'Monthly: $\mu={np.mean(profit_monthly):.3f}, \sigma={np.std(profit_monthly):.3f}$')
plt.hist(profit_quaterly, bins=50, alpha=0.2, color='purple',
         label=f'Quaterly: $\mu={np.mean(profit_quaterly):.3f}, \sigma={np.std(profit_quaterly):.3f}$')

plt.legend(fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('Profit & Loss', fontsize=25)
plt.ylabel('Frequency', fontsize=25)
plt.title('Distribution of P&L with different hedging frequencies', fontsize=25)
plt.show()

In [ ]:
arrays = [profit_daily, profit_weekly, profit_monthly, profit_quaterly]
labels = ['Daily', 'Weekly', 'Monthly', 'Quaterly']
colors = ['green', 'lightblue', 'orange', 'purple']

fig, ax = plt.subplots()

# draw the boxplot with patch_artist to allow filling
bp = ax.boxplot(
    arrays,
    patch_artist=True,
    widths=0.6,
    boxprops=dict(linewidth=1.5),
    medianprops=dict(linewidth=1.5, color='black'),
    whiskerprops=dict(linewidth=1.2),
    capprops=dict(linewidth=1.2)
)

# fill each box with the specified color
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

# add horizontal reference line at y = 0
ax.axhline(0, linestyle='--', linewidth=1, color='gray')

# set x‑ticks, labels and title
ax.set_xticklabels(labels, fontsize=18)
ax.set_ylabel('Profit', fontsize=25)
ax.set_xlabel('Different hedging frequency', fontsize=25)

plt.tight_layout()
plt.show()

In [ ]:
# Example: suppose your five arrays are named arr1 … arr5
arrays = [profit_daily, profit_weekly, profit_monthly, profit_quaterly]
names  = ['Daily', 'Weekly', 'Monthly', 'Quaterly']
alpha  = 0.05  # for 95% confidence

results = {}

for name, arr in zip(names, arrays):
    n    = len(arr)
    mean = np.mean(arr)
    sd   = np.std(arr, ddof=1)             # sample standard deviation
    se   = sd / np.sqrt(n)                 # standard error of the mean
    df   = n - 1
    t_crit = stats.t.ppf(1 - alpha/2, df)  # two‑tailed
    moe  = t_crit * se                     # margin of error
    
    ci_lower = mean - moe
    ci_upper = mean + moe
    
    results[name] = {
        "n": n,
        "mean": mean,
        "std (ddof=1)": sd,
        "95% CI": (ci_lower, ci_upper)
    }

# Display results
for name, stats_dict in results.items():
    print(f"{name}: n={stats_dict['n']}, "
          f"mean={stats_dict['mean']:.4f}, "
          f"std={stats_dict['std (ddof=1)']:.4f}, "
          f"95% CI=[{stats_dict['95% CI'][0]:.4f}, {stats_dict['95% CI'][1]:.4f}]")


## Varying volatility of delta valuation

In [ ]:
def volatility_mismatch(steps, years, start_price, strike_price, rate, volatility, sigma_val, n_sims=1000):
    profit=np.zeros(n_sims)
    for i in range(n_sims):
        B = BlackScholesModel(years, start_price, strike_price, rate, volatility, steps, sigma_val)
        profit[i] = B.create_hedge(steps)
    return profit

In [ ]:
'''Volatility=0.05'''
vol_5 = volatility_mismatch(steps = 365,
    years = 1,
    start_price = 100,
    strike_price = 99,
    rate = 0.06,
    volatility = 0.2,
    sigma_val = 0.05)

'''Volatility=0.1'''
vol_10 = volatility_mismatch(steps = 365,
    years = 1,
    start_price = 100,
    strike_price = 99,
    rate = 0.06,
    volatility = 0.2,
    sigma_val = 0.1)

'''Volatility=0.2'''
vol_20 = volatility_mismatch(steps = 365,
    years = 1,
    start_price = 100,
    strike_price = 99,
    rate = 0.06,
    volatility = 0.2,
    sigma_val = 0.2)

'''Volatility=0.4'''
vol_40 = volatility_mismatch(steps = 365,
    years = 1,
    start_price = 100,
    strike_price = 99,
    rate = 0.06,
    volatility = 0.2,
    sigma_val = 0.4)

'''Volatility=0.5'''
vol_50 = volatility_mismatch(steps = 365,
    years = 1,
    start_price = 100,
    strike_price = 99,
    rate = 0.06,
    volatility = 0.2,
    sigma_val = 0.5)

In [ ]:
plt.figure(figsize=(16,8))
plt.hist(vol_5, bins=50, alpha=0.5, color='orange',
         label=f'5%: $\mu={np.mean(vol_5):.3f}, \sigma={np.std(vol_5):.3f}$')
plt.hist(vol_10, bins=50, alpha=0.5, color='green',
         label=f'10%: $\mu={np.mean(vol_10):.3f}, \sigma={np.std(vol_10):.3f}$')
plt.hist(vol_20, bins=50, alpha=0.5, color='lightblue',
         label=f'20%: $\mu={np.mean(vol_20):.3f}, \sigma={np.std(vol_20):.3f}$')
plt.hist(vol_40, bins=50, alpha=0.5, color='pink',
         label=f'40%: $\mu={np.mean(vol_40):.3f}, \sigma={np.std(vol_40):.3f}$')
plt.hist(vol_50, bins=50, alpha=0.5, color='purple',
         label=f'50%: $\mu={np.mean(vol_50):.3f}, \sigma={np.std(vol_50):.3f}$')

plt.legend(fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('Profit & Loss', fontsize=25)
plt.ylabel('Frequency', fontsize=25)
plt.title('Distribution of P&L with different volatility in delta valuation', fontsize=25)
plt.show()

In [ ]:
arrays = [vol_5, vol_10, vol_20, vol_40, vol_50]
labels = ['5%', '10%', '20%', '40%', '50%']
colors = ['orange', 'green', 'lightblue', 'pink', 'purple']

fig, ax = plt.subplots()

# draw the boxplot with patch_artist to allow filling
bp = ax.boxplot(
    arrays,
    patch_artist=True,
    widths=0.6,
    boxprops=dict(linewidth=1.5),
    medianprops=dict(linewidth=1.5, color='black'),
    whiskerprops=dict(linewidth=1.2),
    capprops=dict(linewidth=1.2)
)

# fill each box with the specified color
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

# add horizontal reference line at y = 0
ax.axhline(0, linestyle='--', linewidth=1, color='gray')

# set x‑ticks, labels and title
ax.set_xticklabels(labels, fontsize=18)
ax.set_ylabel('Profit', fontsize=25)
ax.set_xlabel('Volatility in delta valuation', fontsize=25)

plt.tight_layout()
plt.show()

In [ ]:
# Example: suppose your five arrays are named arr1 … arr5
arrays = [profit_daily, profit_weekly, profit_monthly, profit_quaterly]
names  = ['Daily', 'Weekly', 'Monthly', 'Quaterly']
alpha  = 0.05  # for 95% confidence

results = {}

for name, arr in zip(names, arrays):
    n    = len(arr)
    mean = np.mean(arr)
    sd   = np.std(arr, ddof=1)             # sample standard deviation
    se   = sd / np.sqrt(n)                 # standard error of the mean
    df   = n - 1
    t_crit = stats.t.ppf(1 - alpha/2, df)  # two‑tailed
    moe  = t_crit * se                     # margin of error
    
    ci_lower = mean - moe
    ci_upper = mean + moe
    
    results[name] = {
        "n": n,
        "mean": mean,
        "std (ddof=1)": sd,
        "95% CI": (ci_lower, ci_upper)
    }

# Display results
for name, stats_dict in results.items():
    print(f"{name}: n={stats_dict['n']}, "
          f"mean={stats_dict['mean']:.4f}, "
          f"std={stats_dict['std (ddof=1)']:.4f}, "
          f"95% CI=[{stats_dict['95% CI'][0]:.4f}, {stats_dict['95% CI'][1]:.4f}]")
